<a href="https://colab.research.google.com/github/mudogruer/transformers/blob/main/chess_prediction_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAN AUTO-REGRESSION MODELS GENERATE A CHESS GAME


I though that an decoder language model can create a logical chess game while I am precticing on NLP. Then I want to create a model to see the result.
Let's see some funny results and learn about decoders.

First download libraries that we need.

In [ ]:
!pip install -q -U accelerate datasets transformers

I've found a dataset on HuggingFace which includes 360k chess game. But I cannot use this amount of game because of RAM capacity.
Max. %5 of it can be used regarding my trying.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('mlabonne/chessllm',split='train[95%:]')


Upload the data and see what we have here.

In [ ]:
print(dataset)
print(dataset[0])

DatasetDict({
    train: Dataset({
        features: ['average_elo', 'text'],
        num_rows: 16230
    })
})


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"

The last 3 character is about the results. We dont need them here. Lets create a column for them.

In [ ]:
def extract_scores(example):
    # Extract the last 3 characters from the 'transcript' field
    example['scores'] = example['transcript'][-3:]
    example['transcript'] = example['transcript'][:-3]
    return example

# Apply the function to each example in the dataset
dataset = dataset.map(lambda examples: extract_scores(examples), batched=False)

In [ ]:
dataset

Now, I want to record scores in another column as integer to analyze.

In [ ]:
def create_label_column(example):
    # Extract the last 3 characters from the 'transcript' field
    if example['scores'] == '1-0':
        example['label'] = 1
    elif example['scores'] == '1/2':
        example['label'] = 0
    elif example['scores'] == '0-1':
        example['label'] = -1
    else:
        example['label']= None
    return example

# Apply the function to each example in the dataset
dataset = dataset.map(lambda examples: create_label_column(examples), batched=False)

In [ ]:
dataset

In [ ]:
from datasets import *

dataset = DatasetDict({"train":dataset})

In [ ]:
dataset["train"].features

Everthing looks good. Now lets make them dataframe and go deeper

In [ ]:
import pandas as ps

dataset.set_format(type="pandas")

In [ ]:
df= dataset["train"][:]
df.head(10)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df["label"].value_counts(ascending = True).plot.barh()
plt.title("Frequency of Scores")
plt.show()

Whites have advantages. That makes sense.

In [ ]:
df["tokens_per_game"] = df["transcript"].str.split().apply(len)
df.boxplot("tokens_per_game", by = "label", grid = False,showfliers=False, color = "black")
plt.suptitle("")
plt.xlabel("")
plt.show()

Data looks ok. Goodbye dataframe

In [ ]:
dataset.reset_format()
dataset

We want to use decoder model. So, we dont need label or any classification thing. We cannot rule this model. He is the boss here. <br>
Because of this, we need to remove scores.

In [ ]:
dataset = dataset["train"].rename_column("transcript","text")
dataset

In [ ]:
dataset = DatasetDict({"train":dataset})

In [ ]:
dataset = dataset.remove_columns("scores")
dataset = dataset.remove_columns("label")
dataset

In [ ]:
dataset["train"].features

We understand data. But model cannot speak this language. Lets speak in his language. <br>
Let the tokenization begin!

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"],truncation=True)

Of course, I will push the hub this model. Maybe someone inspire of it :P

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

I chose to use GPT2 because why not. We define training arguments and other requirements.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Assume dataset_encoded is correctly tokenized and prepared
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="adamw_hf",  # Corrected optimizer name
    save_strategy="steps",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-3,
    weight_decay=0.001,
    max_steps=50,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


The training was over and I was able to train 50 steps due to physical disabilities. Let's see what a little trained decoder model can do.

In [ ]:
prompt = "Generate a chess game begin with these moves:"+"1. e4 e5 2. d4 d5 3. exd5 exd4 4. Nf3 Bf5"  # Your chess move sequence

# Tokenize the prompt to input IDs
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Generate text using the model. Adjust the parameters as needed.
max_length = input_ids.shape[1] + 25  # You can adjust the length of the prediction
temperature = 0.5  # Controls the randomness: lower means less random
num_return_sequences = 1  # Number of sequences to generate

# Generate predictions
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_length,
    temperature=temperature,
    top_k=5,
    top_p=0.95,
    repetition_penalty=0.5,
    do_sample=True,
    num_return_sequences=num_return_sequences,
)

# Decode the generated sequences to text
generated_sequences = []
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(tokenizer.eos_token) if tokenizer.eos_token else None]

    # Add the prompt at the beginning of the sequence. Remove the prompt from the generated sequence if not desired.
    total_sequence = (
        prompt + text[len(tokenizer.decode(input_ids[0], clean_up_tokenization_spaces=True)) :]
    )

    generated_sequences.append(total_sequence)

for generated_sequence in generated_sequences:
    print(generated_sequence)


In [ ]:
# my_finetuned_model = "gpt2-chess-move-prediction"

# trainer.model.push_to_hub(my_finetuned_model)

## CONCLUSION

By its nature, there is no such thing as unreasonable as expecting logical results from decoder models. But they say curiosity killed the cat, so I was curious and tried it. While 1 or 2 moves may be logical, the other moves consist of completely illogical moves. Still, it looked real enough to fool someone who didn't know chess.

God knows how much more logical the answers would be if I could use the entire data set and run more epochs. Of course, something can be found by playing on the educational arguments.

What this idle study contributed to me was to better understand the working logic of decoder models. It would be better if we leave the chess job to reinforcement learning algorithms.